# Pose Classifier

## Import Libraries

In [14]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

from calculations import calc_length, calc_angle, calc_line_orientation, get_xy
from classifier import PoseDataset, PoseClassifier

## Load Dataset

In [15]:
df = pd.read_csv('labeled_keypoints.csv')
df.describe()

,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,...,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,...,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,470.005604,477.117375,498.117494,456.888212,446.769159,455.892383,524.853798,480.911359,398.498385,478.539675,...,405.292769,1137.041367,586.762294,1445.536885,362.383013,1413.312563,558.093760,1632.971048,340.942543,1627.718210
std,107.682845,132.217978,112.730499,129.851882,102.780046,129.459627,151.597572,163.142625,121.542621,162.496531,...,55.081617,52.204326,88.432818,224.460151,75.754583,279.930584,159.987250,468.071637,101.760031,466.874618
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,115.050911,868.854553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,475.145882,489.668686,503.434616,471.778778,450.713722,470.203659,550.600983,515.412598,417.954514,512.099304,...,407.733833,1123.277740,588.465271,1450.086884,366.208900,1443.138306,599.470230,1761.833862,344.224625,1754.219421
50%,487.820038,505.308167,517.501434,485.195404,463.863052,485.228455,563.897583,526.270752,430.053787,526.490601,...,416.778687,1127.515503,599.107971,1453.989258,374.385773,1448.028503,603.577972,1773.181152,353.313385,1763.323608
75%,503.923279,511.493301,533.297485,490.979851,480.338608,490.964134,574.640717,536.424194,446.651604,533.734177,...,428.538528,1134.621735,609.885666,1468.060699,392.443085,1460.288635,607.290131,1778.319977,380.174194,1772.807831
max,611.382141,700.876953,619.075867,676.828308,585.919617,674.866943,669.822754,726.855835,531.884583,722.057373,...,503.083649,1305.705078,677.237427,1860.718140,451.678711,1842.842407,690.043640,1877.032104,457.488098,1874.398804


## Feature Engineering

### Lengths

In [16]:
# head
df['eye_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_eye'),
        get_xy(row, 'right_eye')
    ),
    axis=1
)

df['right_eye_nose_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_eye'),
        get_xy(row, 'nose'),
    ),
    axis=1
)
df['left_eye_nose_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_eye'),
        get_xy(row, 'nose'),
    ),
    axis=1
)

df['right_eye_ear_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_eye'),
        get_xy(row, 'right_ear'),
    ),
    axis=1
)
df['left_eye_ear_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_eye'),
        get_xy(row, 'left_ear'),
    ),
    axis=1
)

df['right_ear_shoulder_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_ear'),
        get_xy(row, 'right_shoulder'),
    ),
    axis=1
)
df['left_ear_shoulder_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_ear'),
        get_xy(row, 'left_shoulder'),
    ),
    axis=1
)

df['shoulder_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'left_shoulder'),
    ),
    axis=1
)

# torso
df['right_upper_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'right_elbow'),
    ),
    axis=1
)
df['left_upper_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'left_elbow'),
    ),
    axis=1
)

df['right_forearm_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_elbow'),
        get_xy(row, 'right_wrist'),
    ),
    axis=1
)
df['left_forearm_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_elbow'),
        get_xy(row, 'left_wrist'),
    ),
    axis=1
)

# legs
df['hip_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_hip'),
        get_xy(row, 'left_hip'),
    ),
    axis=1
)

df['right_thigh_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_hip'),
        get_xy(row, 'right_knee'),
    ),
    axis=1
)
df['left_thigh_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_hip'),
        get_xy(row, 'left_knee'),
    ),
    axis=1
)

df['right_shin_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'right_knee'),
        get_xy(row, 'right_ankle'),
    ),
    axis=1
)
df['left_shin_width'] = df.apply(
    lambda row: calc_length(
        get_xy(row, 'left_knee'),
        get_xy(row, 'left_ankle'),
    ),
    axis=1
)

### Angles

In [17]:
# head
df['nose_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_eye'),
        get_xy(row, 'nose'),
        get_xy(row, 'right_eye'),
    ),
    axis=1
)

df['right_ear_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'right_eye'),
        get_xy(row, 'right_ear'),
        get_xy(row, 'right_shoulder'),
    ),
    axis=1
)
df['left_ear_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_eye'),
        get_xy(row, 'left_ear'),
        get_xy(row, 'left_shoulder'),
    ),
    axis=1
)

# torso
df['right_shoulder_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'right_hip'),
    ),
    axis=1
)
df['left_shoulder_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'left_hip'),
    ),
    axis=1
)

df['right_elbow_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'right_elbow'),
        get_xy(row, 'right_wrist'),
    ),
    axis=1
)
df['left_elbow_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'left_elbow'),
        get_xy(row, 'left_wrist'),
    ),
    axis=1
)

# legs
df['right_hip_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_hip'),
        get_xy(row, 'right_hip'),
        get_xy(row, 'right_knee'),
    ),
    axis=1
)
df['left_hip_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'right_hip'),
        get_xy(row, 'left_hip'),
        get_xy(row, 'left_knee'),
    ),
    axis=1
)

df['right_knee_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'right_hip'),
        get_xy(row, 'right_knee'),
        get_xy(row, 'right_ankle'),
    ),
    axis=1
)
df['left_knee_angle'] = df.apply(
    lambda row: calc_angle(
        get_xy(row, 'left_hip'),
        get_xy(row, 'left_knee'),
        get_xy(row, 'left_ankle'),
    ),
    axis=1
)

### Orientation

In [18]:
# head
df['right_ear_eye_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'right_ear'),
        get_xy(row, 'right_eye'),
    ),
    axis=1
)
df['left_ear_eye_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'left_ear'),
        get_xy(row, 'left_eye'),
    ),
    axis=1
)

# torso
df['right_shoulder_ear_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'right_ear'),
    ),
    axis=1
)
df['left_shoulder_ear_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'left_ear'),
    ),
    axis=1
)

df['right_shoulder_elbow_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'right_shoulder'),
        get_xy(row, 'right_elbow'),
    ),
    axis=1
)
df['left_shoulder_elbow_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'left_shoulder'),
        get_xy(row, 'left_elbow'),
    ),
    axis=1
)

df['right_elbow_wrist_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'right_elbow'),
        get_xy(row, 'right_wrist'),
    ),
    axis=1
)
df['left_elbow_wrist_dir'] = df.apply(
    lambda row: calc_line_orientation(
        get_xy(row, 'left_elbow'),
        get_xy(row, 'left_wrist'),
    ),
    axis=1
)

### View Results

In [19]:
df.columns

Index(['nose_x', 'nose_y', 'left_eye_x', 'left_eye_y', 'right_eye_x',
       'right_eye_y', 'left_ear_x', 'left_ear_y', 'right_ear_x', 'right_ear_y',
       'left_shoulder_x', 'left_shoulder_y', 'right_shoulder_x',
       'right_shoulder_y', 'left_elbow_x', 'left_elbow_y', 'right_elbow_x',
       'right_elbow_y', 'left_wrist_x', 'left_wrist_y', 'right_wrist_x',
       'right_wrist_y', 'left_hip_x', 'left_hip_y', 'right_hip_x',
       'right_hip_y', 'left_knee_x', 'left_knee_y', 'right_knee_x',
       'right_knee_y', 'left_ankle_x', 'left_ankle_y', 'right_ankle_x',
       'right_ankle_y', 'label', 'eye_width', 'right_eye_nose_width',
       'left_eye_nose_width', 'right_eye_ear_width', 'left_eye_ear_width',
       'right_ear_shoulder_width', 'left_ear_shoulder_width', 'shoulder_width',
       'right_upper_width', 'left_upper_width', 'right_forearm_width',
       'left_forearm_width', 'hip_width', 'right_thigh_width',
       'left_thigh_width', 'right_shin_width', 'left_shin_width', 'nos

In [20]:
len(df.columns)

71

## Training

### Normalizing

In [21]:
numeric_cols = df.select_dtypes(include='number').columns
scaler = StandardScaler()
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

### Setup Dataset

In [22]:
# label encoding
label_encoder = LabelEncoder()
label_encoder.fit(['none', 'head_up', 'head_left', 'head_right', 'head_down', 'crouch', 'walk'])

# data loader
dataset = PoseDataset(df, label_encoder)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

### Create Model

In [23]:
model = PoseClassifier(input_dim=70, num_classes=len(label_encoder.classes_))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### Train Model

In [24]:
NUM_EPOCHS = 100
model.train()
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    avg_loss = running_loss / len(dataloader)
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {avg_loss:.4f}')

Epoch [1/100], Loss: 1.9873
Epoch [2/100], Loss: 1.8053
Epoch [3/100], Loss: 1.5986
Epoch [4/100], Loss: 1.3706
Epoch [5/100], Loss: 1.1043
Epoch [6/100], Loss: 0.9726
Epoch [7/100], Loss: 0.8295
Epoch [8/100], Loss: 0.8315
Epoch [9/100], Loss: 0.6624
Epoch [10/100], Loss: 0.5847
Epoch [11/100], Loss: 0.5581
Epoch [12/100], Loss: 0.4773
Epoch [13/100], Loss: 0.4223
Epoch [14/100], Loss: 0.3461
Epoch [15/100], Loss: 0.3152
Epoch [16/100], Loss: 0.2878
Epoch [17/100], Loss: 0.2431
Epoch [18/100], Loss: 0.3322
Epoch [19/100], Loss: 0.2165
Epoch [20/100], Loss: 0.2238
Epoch [21/100], Loss: 0.1767
Epoch [22/100], Loss: 0.1628
Epoch [23/100], Loss: 0.1691
Epoch [24/100], Loss: 0.1475
Epoch [25/100], Loss: 0.1340
Epoch [26/100], Loss: 0.1428
Epoch [27/100], Loss: 0.1216
Epoch [28/100], Loss: 0.1227
Epoch [29/100], Loss: 0.1022
Epoch [30/100], Loss: 0.0976
Epoch [31/100], Loss: 0.0927
Epoch [32/100], Loss: 0.0934
Epoch [33/100], Loss: 0.0842
Epoch [34/100], Loss: 0.0790
Epoch [35/100], Loss: 0

## Export

In [25]:
torch.save(model.state_dict(), 'pose-classifier.pt')
joblib.dump(scaler, 'scaler.pkl');